In [106]:
import numpy as np
import pandas as pd
import plotly 
import wget
from urllib.request import urlopen
import json


In [35]:
#GeoJSON file from france-geojson.gregoiredavid.fr
url = "https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/departements-version-simplifiee.geojson"

with urlopen(url) as response:
    departments = json.load(response)

metropolitan_dep = [departments['features'][i]['properties']['code'] for i in range(len(departments['features']))]

In [97]:
df_inc = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/19a91d64-3cd3-42fc-9943-d635491a4d76',delimiter=';')
df_test = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/3c18e242-7d45-44f2-ac70-dee78a38ee1c',delimiter=';')

#Preprocessing and calculating incidence rate
df_inc.set_index(df_inc.index.rename('og_index'),inplace = True)
df_inc['dep'] = df_inc['dep'].astype('string')
df_inc['jour']= df_inc['jour'].astype('string')
df_inc = df_inc[df_inc['dep'].apply(lambda k: k in metropolitan_dep)]
df_inc = df_inc[df_inc['cl_age90'] == 0].drop('cl_age90',axis = 1)
df_inc['incidence'] = df_inc['P']/df_inc['pop']*100000
df_inc = df_inc.groupby('dep').rolling(7).sum().join(df_inc[['dep','jour']],how = 'inner')

#Preprocessing and calculating test positivity rate
df_test['dep'] = df_test['dep'].astype('string')
df_test = df_test[df_test['dep'].apply(lambda k: k in metropolitan_dep)]
df_test['jour'] = df_test['semaine_glissante'].apply(lambda k: k[-10:]).astype('string')
df_test['positivity'] = df_test['P']/df_test['T']*100

df_inc

C:\Users\vince\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


P        pop   incidence dep        jour
dep og_index                                               
01  10           NaN        NaN         NaN  01  2020-05-13
    21           NaN        NaN         NaN  01  2020-05-14
    32           NaN        NaN         NaN  01  2020-05-15
    43           NaN        NaN         NaN  01  2020-05-16
    54           NaN        NaN         NaN  01  2020-05-17
...              ...        ...         ...  ..         ...
95  401235    2435.0  8738478.0  195.056851  95  2021-05-23
    401246    1948.0  8738478.0  156.045481  95  2021-05-24
    401257    1947.0  8738478.0  155.965375  95  2021-05-25
    401268    1935.0  8738478.0  155.004109  95  2021-05-26
    401279    1909.0  8738478.0  152.921367  95  2021-05-27

[36480 rows x 5 columns]

In [121]:
data_slider = []
steps = []
days = df_inc['jour'].unique()[-4:]

for i,day in enumerate(days):

    data_one_day = dict(type='choropleth',
                        colorscale='rdylgn_r',
                        geojson=departments,
                        z=df_inc[df_inc['jour']==day]['incidence'],
                        locations=df_inc[df_inc['jour']==day]['dep'],
                        text=metropolitan_dep,
                        featureidkey='properties.code')
    data_slider.append(data_one_day)

    step = dict(method='restyle',
                args=['visible', [False] * len(days)],
                label=day) # label to be displayed for each step (year)
    step["args"][1][i] = True
    steps.append(step)

slider = [dict(active=0, pad={"t": 1}, steps=steps)]  

layout = dict(geo={'projection':{'type':'natural earth'},'fitbounds':'geojson'},
            sliders = slider)


fig = dict(data=data_slider, layout=layout) 

plotly.offline.iplot(fig)

ValueError: 
    Invalid value of type 'builtins.str' received for the 'colorscale' property of choropleth
        Received value: 'ylrdgr_r'

    The 'colorscale' property is a colorscale and may be
    specified as:
      - A list of colors that will be spaced evenly to create the colorscale.
        Many predefined colorscale lists are included in the sequential, diverging,
        and cyclical modules in the plotly.colors package.
      - A list of 2-element lists where the first element is the
        normalized color level value (starting at 0 and ending at 1),
        and the second item is a valid color string.
        (e.g. [[0, 'green'], [0.5, 'red'], [1.0, 'rgb(0, 0, 255)']])
      - One of the following named colorscales:
            ['aggrnyl', 'agsunset', 'algae', 'amp', 'armyrose', 'balance',
             'blackbody', 'bluered', 'blues', 'blugrn', 'bluyl', 'brbg',
             'brwnyl', 'bugn', 'bupu', 'burg', 'burgyl', 'cividis', 'curl',
             'darkmint', 'deep',

In [101]:
step

{'step': {'method': 'restyle',
  'args': ['visible', [False, False, False, False]],
  'label': '2021-05-24'}}